# Grover's Search Algorithm

We want to find a specific item in an unsorted list of $m = 2^n$ items.

Define $f(x) = 1$ `if` $x = x_0$ `else` 0

In [1]:
from quantum import *
from math import log, ceil

TARGET = 0b101

def func(x):
    return 1 if x == TARGET else 0

The input register is $n$-qubits: $\left|00...0\right\rangle$

The output register is $\left|0\right\rangle$

In [2]:
n = ceil(log(TARGET, 2))
m = 2 ** n

state = Q(kron(*(zero for _ in range(n)), zero))
print(state)

[[1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


Apply `HX` to the output register.

In [3]:
state = state.apply_gate(X, 0)
state = state.apply_gate(H, 0)
print(state)

[[ 0.70710678+0.j]
 [-0.70710678+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]
 [ 0.00000000+0.j]]


Apply `H`$^{\otimes n}$ to the input register.

In [4]:
# input register: qubits 1 to n
#input_hadamard = kron(*(I if q == 0 else H for q in reversed(range(n+1))))
#state = state.apply_unitary(input_hadamard)
for i in range(1, n+1):
    state = state.apply_gate(H, i)
print(state)

[[ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]]


Apply $f(x)$.

In [5]:
state = state.apply_func(func)
print(state)

[[ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]
 [ 0.25+0.j]
 [-0.25+0.j]]


Apply inversion about the mean.

In [6]:
N = m
# this definition of D (the diffusion matrix aka. inversion about the mean)
# is given in Grover's original paper
P = 1/N * np.ones((N,N))
D = -np.eye(N) + 2*P
# I assume we leave the output register untouched (hence the final I)
D = kron(D, I)
state = state.apply_unitary(D)
print(state)

[[ 0.125+0.j]
 [-0.125+0.j]
 [ 0.125+0.j]
 [-0.125+0.j]
 [ 0.125+0.j]
 [-0.125+0.j]
 [ 0.125+0.j]
 [-0.125+0.j]
 [ 0.125+0.j]
 [-0.125+0.j]
 [ 0.625+0.j]
 [-0.625+0.j]
 [ 0.125+0.j]
 [-0.125+0.j]
 [ 0.125+0.j]
 [-0.125+0.j]]


In [7]:
ITERATIONS = 1000
for iteration in range(ITERATIONS):
    state = state.apply_func(func)
    state = state.apply_unitary(D)
print(state)

[[ 0.08375616+0.j]
 [-0.08375616+0.j]
 [ 0.08375616+0.j]
 [-0.08375616+0.j]
 [ 0.08375616+0.j]
 [-0.08375616+0.j]
 [ 0.08375616+0.j]
 [-0.08375616+0.j]
 [ 0.08375616+0.j]
 [-0.08375616+0.j]
 [ 0.67148666+0.j]
 [-0.67148666+0.j]
 [ 0.08375616+0.j]
 [-0.08375616+0.j]
 [ 0.08375616+0.j]
 [-0.08375616+0.j]]


In [8]:
TRIALS = 4000
outcomes = {}
for trial in range(TRIALS):
    measurement = state.measure() >> 1 # measure state and chop off the output register
    outcomes[measurement] = outcomes.get(measurement, 0) + 1
    
qubits = state.num_qubits
for outcome in sorted(outcomes.keys()):
    occurrences = outcomes[outcome]
    rate = 100 * occurrences / TRIALS
    print("Measured {outcome:2} {outcome:0{width}b} ({rate:5.2f}%)".format(outcome=outcome, width=qubits, rate=rate))

Measured  0 0000 ( 1.52%)
Measured  1 0001 ( 1.27%)
Measured  2 0010 ( 1.27%)
Measured  3 0011 ( 1.30%)
Measured  4 0100 ( 1.27%)
Measured  5 0101 (90.28%)
Measured  6 0110 ( 1.40%)
Measured  7 0111 ( 1.68%)
